#### 전주 순번 및 전주와 지선주 갯 수 계산

In [1]:
import pandas as pd
from datetime import datetime 

from freeman.aiddd.data_manager import read_data, write_data

#### 데이터 불러오기

In [2]:
_start_time = datetime.now()

# 4차 전처리: 공사비 + 전주 + 전선 데이터 
_df_cons = read_data(
    '1d-merge-cons-pole-line', process_seq='4th', dtype={'cons_id': str}
)

# 4차 전처리: 전주 데이터(전선 정보를 이용해 순서 정하고 좌표 지정하기 위해)
_df_pole = read_data(
    '1c-preprocessed-pole', process_seq='4th', dtype={'cons_id': str}
)

# 4차 전처리: 전선 데이터
_df_line = read_data(
    '1d-preprocessed-line', process_seq='4th', dtype={'cons_id': str}
)

print(
    f'Total Elapsed Time for Data Load: {datetime.now() - _start_time}\n'
    f'Preprocessed Data(cons) Shape: {_df_cons.shape}\n'
    f'Preprocessed Data(pole) Shape: {_df_pole.shape}\n'
    f'Preprocessed Data(line) Shape: {_df_line.shape}'
)

Total Elapsed Time for Data Load: 0:00:00.463533
Preprocessed Data(cons) Shape: (14728, 81)
Preprocessed Data(pole) Shape: (26920, 20)
Preprocessed Data(line) Shape: (29704, 51)


In [3]:
df_cons, df_pole, df_line = _df_cons.copy(), _df_pole.copy(), _df_line.copy()

#### DF_POLE dict생성

In [4]:
dict_pole = df_pole[['comp_id', 'x', 'y']]\
    .set_index('comp_id').T.to_dict('list')

# 확인
dict_pole[df_pole.iloc[1].comp_id]

[128.393165784208, 36.8303489085259]

#### 공사번호별 전주 순서 계산

In [7]:
# 공사번호별 전주 및 지선주 갯 수 계산
def computed_pole_counts(df_curr_pole, df_curr_line):
    # 지선주 갯 수: df_temp.shape[0]
    # 실제 전주 갯 수: pole_cnts - 지선주 갯 수
    df_temp = df_curr_pole[
        ~df_curr_pole.comp_id.isin(df_curr_line.comp_id) &
        ~df_curr_pole.comp_id.isin(df_curr_line.from_comp_id)
    ]
    curr_total_pole_count = df_curr_pole.shape[0]
    curr_support_pole_count = df_temp.shape[0]
    return [
        curr_total_pole_count - curr_support_pole_count,
        curr_support_pole_count
    ]

In [49]:
# 공사번호별 전주 순서계산 함수
def computed_pole_seqs(paths, df_curr_pole, df_curr_line):
    # 전선에 연결된 전산화번호 값들
    comp_id_values = df_curr_line.comp_id.tolist()
    # 전선에 연결된 전원측전산화번호 값들
    from_comp_id_values = df_curr_line.from_comp_id.tolist()
    # 전주간 거리
    span_values = df_curr_line.span.tolist()
    # 전주 자체 전산화번호 값들
    pole_comp_id_values = df_curr_pole.comp_id.tolist()
    # 전원측전산화번호에만 있는 값들(이 전주가 전체 경로의 시작점)
    only_from_comp_id_values = [
        item for item in from_comp_id_values if item not in comp_id_values
    ]
    
    # print(
    #     f'pole: {pole_comp_id_values}\n'
    #     f'comp: {comp_id_values}\n'
    #     f'from: {from_comp_id_values}{only_from_comp_id_values}'
    # )
    
    # 연속한 두 전주의 기설 여부 체크
    # 이런 경우는 전주작업 없이 전선작업만 한 경우로 예외 처리
    before_is_already = False
    # 한 전주에서 2번 이상 분기한 경우 체크
    # 이런 경우는 정상적인 경우가 아니기 때문에 예외 처리
    processed_comp_id = []
    # 두 전주가 전선으로 연결되지 않고 끈겨 있는 횟 수 
    # 이 값은 모델 속성값으로 사용할 예정
    disconnected_count = 0
    
    # 회전 횟 수를 1 추가해 비정상 종료 시키면서 마지막 next_span값에 0 추가
    curr_loop_count = len(from_comp_id_values) + 1
    sub_loop_count = 0
    is_exception = False
    for comp_id in only_from_comp_id_values:
        # 하나의 전주(전산화번호)에서 여러 전주로 연결되는 공사(제외대상)
        if comp_id in processed_comp_id:
            is_exception = True
            break
        processed_comp_id.append(comp_id)
        
        next_comp_id = comp_id
        for _ in range(curr_loop_count - sub_loop_count):
            # 현재 전주의 기설(1) 또는 신설(0) 여부 판단
            is_already = next_comp_id not in pole_comp_id_values
            # print(f'**{is_already}:{next_comp_id}:{pole_comp_id_values}')
            if is_already and before_is_already:
                is_exception = True
                break
            before_is_already = is_already
            
            # 현 전주의 좌표값 가져오기
            xy = dict_pole.get(next_comp_id, [0, 0])
            
            try:
                next_comp_id_index = from_comp_id_values.index(next_comp_id)
                next_span = span_values[next_comp_id_index]
                paths.extend([
                    is_already, next_comp_id, xy[0], xy[1], next_span
                ])
                next_comp_id = comp_id_values[next_comp_id_index]
            except ValueError as ve:
                # 전주와 전주 사이에 연결이 끈어진 경우
                next_span = 0
                disconnected_count += 1
                paths.extend([
                    is_already, next_comp_id, xy[0], xy[1], next_span
                ])
                break
                
            sub_loop_count += 1
    
    # 전주와 전주 사이가 끈어진 경우 제외
    if disconnected_count > 1:
        is_exception = True
            
    return is_exception, disconnected_count

In [50]:
# 공사번호별 전주 경로계산 함수
def computed_pole_paths(cons_id):
    # 공사번호 추가
    pole_paths = [cons_id]
    
    df_curr_pole = df_pole[df_pole.cons_id == cons_id]
    df_curr_line = df_line[df_line.cons_id == cons_id]
    
    # 전주 및 지선주 갯 수 추가
    pole_paths.extend(computed_pole_counts(df_curr_pole, df_curr_line))
    
    # 전주 순서 구하기
    is_exception, disconnected_count = \
        computed_pole_seqs(pole_paths, df_curr_pole, df_curr_line)
        
    return is_exception, disconnected_count, pole_paths

In [51]:
# 공사번호 리스트
unique_cons_ids = df_cons.cons_id.unique()
# 공사번호별 전주 순번 등 경로정보
cons_id_pole_paths = []
# 해당 공사에서 전선이 끈긴 횟 수
cons_id_disconnected_counts = []

for cons_id in unique_cons_ids:
    # print(f'{cons_id}:')
    is_exception, disconnected_count, pole_paths = \
        computed_pole_paths(cons_id)
    if is_exception is not True:
        cons_id_pole_paths.append(pole_paths)
        cons_id_disconnected_counts.append(disconnected_count)

In [54]:
MAX_POLE_COUNT = 11
column_names = ['cons_id', 'real_pole_cnts', 'support_pole_cnts']
for idx in range(MAX_POLE_COUNT):
    column_names += [
        f'pole{idx+1}_is_already', f'pole{idx+1}_comp_id',
        f'pole{idx+1}_x', f'pole{idx+1}_y', f'pole{idx+1}_span'
    ]

In [55]:
df_pole_paths = pd.DataFrame(cons_id_pole_paths, columns=column_names)

In [56]:
df_merge = pd.merge(
    df_cons, df_pole_paths,
    left_on='cons_id', right_on='cons_id', how='right'
)

In [59]:
write_data('1e-merge-cons-pole-line-path', df_merge, process_seq='4th')

In [60]:
[col for col in df_merge.columns]

['cons_id',
 'total_cons_cost',
 'cont_cap',
 'last_mod_date',
 'year',
 'month',
 'day',
 'dayofweek',
 'dayofyear',
 'last_mod_eid',
 'eid_code',
 'eid_code_number',
 'eid_number',
 'office_name',
 'office_number',
 'line_cnts',
 'pole_cnts',
 'sl_cnts',
 'pole_shape_G',
 'pole_shape_O',
 'pole_shape_V',
 'pole_type_1',
 'pole_type_B',
 'pole_type_C',
 'pole_type_E',
 'pole_type_H',
 'pole_type_M',
 'pole_spec_6.0',
 'pole_spec_8.0',
 'pole_spec_10.0',
 'pole_spec_11.0',
 'pole_spec_12.0',
 'pole_spec_14.0',
 'pole_spec_16.0',
 'span',
 'line_length',
 'wiring_scheme_13',
 'wiring_scheme_43',
 'line_type_AO',
 'line_type_C1',
 'line_type_C2',
 'line_type_C4',
 'line_type_D2',
 'line_type_D4',
 'line_type_EW',
 'line_type_OC',
 'line_type_OW',
 'line_spec_3.2',
 'line_spec_22.0',
 'line_spec_25.0',
 'line_spec_32.0',
 'line_spec_35.0',
 'line_spec_38.0',
 'line_spec_58.0',
 'line_spec_60.0',
 'line_spec_70.0',
 'line_spec_100.0',
 'line_spec_120.0',
 'line_spec_150.0',
 'line_phase_1'

In [64]:
column_unique_counts = df_merge.nunique()

In [72]:
df_merge[df_merge.pole5_is_already == True].shape

(24, 138)